# Station Mapping

## Required Packages and Setup

In [29]:
#uncomment and run the below line to install folium, which we are using for mapping
#%conda install --name metis -c conda-forge folium

In [30]:
#Required Packages
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta
import urllib.request
import matplotlib.pyplot as plt
import pickle
import folium
from folium import plugins

In [31]:
#Setup Configs
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)
pd.set_option('display.precision', 3)

## Data Import

In [32]:
with open('pickle/station_lookup.pickle','rb') as read_file: #generated in stop_id_matchup.ipynb
    station_lookup = pickle.load(read_file)
station_lookup.head()

,C/A,STATION,stop_id,Stop Name,GTFS Latitude,GTFS Longitude,Borough,Daytime Routes
0,A002,59 ST,R11,Lexington Av/59 St,40.763,-73.967,M,N W R
1,A006,5 AV/59 ST,R13,5 Av/59 St,40.765,-73.973,M,N W R
2,A007,5 AV/59 ST,R13,5 Av/59 St,40.765,-73.973,M,N W R
3,A010,57 ST-7 AV,R14,57 St - 7 Av,40.765,-73.981,M,N Q R W
4,A011,57 ST-7 AV,R14,57 St - 7 Av,40.765,-73.981,M,N Q R W


In [33]:
with open('pickle/mta_data_intra.pickle','rb') as read_file: #generated in scrape_clean.ipynb
    mta_data_intra = pickle.load(read_file)
mta_data_intra.head()

,C/A,UNIT,SCP,STATION,LINENAME,DATETIME,DATE,TIME,TIME_DELTA,ENTRIES_DELTA,EXITS_DELTA,TOTAL_DELTA
0,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30 00:00:00,2019-03-30,00:00:00,04:00:00,20.0,8.0,28.0
1,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30 04:00:00,2019-03-30,04:00:00,04:00:00,23.0,46.0,69.0
2,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30 08:00:00,2019-03-30,08:00:00,04:00:00,107.0,88.0,195.0
3,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30 12:00:00,2019-03-30,12:00:00,04:00:00,237.0,71.0,308.0
4,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30 16:00:00,2019-03-30,16:00:00,04:00:00,345.0,56.0,401.0


In [34]:
with open('pickle/mta_data_daily.pickle','rb') as read_file: #generated in scrape_clean.ipynb
    mta_data_daily = pickle.load(read_file)
mta_data_daily.head()

,C/A,UNIT,SCP,STATION,LINENAME,DATE,TIME_DELTA,ENTRIES_DELTA,EXITS_DELTA,TOTAL_DELTA
0,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30,1 days,893.0,299.0,1192.0
1,A002,R051,02-00-00,59 ST,NQR456W,2019-03-31,1 days,571.0,228.0,799.0
2,A002,R051,02-00-00,59 ST,NQR456W,2019-04-02,1 days,1593.0,554.0,2147.0
3,A002,R051,02-00-00,59 ST,NQR456W,2019-04-03,1 days,1652.0,424.0,2076.0
4,A002,R051,02-00-00,59 ST,NQR456W,2019-04-04,1 days,1638.0,511.0,2149.0


## Merging Coordinate Data

In [35]:
mta_data_intra = pd.merge(mta_data_intra,station_lookup[['C/A','GTFS Latitude','GTFS Longitude']],on='C/A',how='left')
mta_data_intra.head()

,C/A,UNIT,SCP,STATION,LINENAME,DATETIME,DATE,TIME,TIME_DELTA,ENTRIES_DELTA,EXITS_DELTA,TOTAL_DELTA,GTFS Latitude,GTFS Longitude
0,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30 00:00:00,2019-03-30,00:00:00,04:00:00,20.0,8.0,28.0,40.763,-73.967
1,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30 04:00:00,2019-03-30,04:00:00,04:00:00,23.0,46.0,69.0,40.763,-73.967
2,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30 08:00:00,2019-03-30,08:00:00,04:00:00,107.0,88.0,195.0,40.763,-73.967
3,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30 12:00:00,2019-03-30,12:00:00,04:00:00,237.0,71.0,308.0,40.763,-73.967
4,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30 16:00:00,2019-03-30,16:00:00,04:00:00,345.0,56.0,401.0,40.763,-73.967


In [36]:
mta_data_daily = pd.merge(mta_data_daily,station_lookup[['C/A','GTFS Latitude','GTFS Longitude']],on='C/A',how='left')
mta_data_daily.head()

,C/A,UNIT,SCP,STATION,LINENAME,DATE,TIME_DELTA,ENTRIES_DELTA,EXITS_DELTA,TOTAL_DELTA,GTFS Latitude,GTFS Longitude
0,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30,1 days,893.0,299.0,1192.0,40.763,-73.967
1,A002,R051,02-00-00,59 ST,NQR456W,2019-03-31,1 days,571.0,228.0,799.0,40.763,-73.967
2,A002,R051,02-00-00,59 ST,NQR456W,2019-04-02,1 days,1593.0,554.0,2147.0,40.763,-73.967
3,A002,R051,02-00-00,59 ST,NQR456W,2019-04-03,1 days,1652.0,424.0,2076.0,40.763,-73.967
4,A002,R051,02-00-00,59 ST,NQR456W,2019-04-04,1 days,1638.0,511.0,2149.0,40.763,-73.967


In [37]:
mta_data_daily_station = (mta_data_daily
                          .groupby(['STATION','GTFS Latitude','GTFS Longitude','DATE'])['ENTRIES_DELTA','EXITS_DELTA','TOTAL_DELTA']
                          .sum()
                          .reset_index()
                         )

mta_data_daily_station['circ_value'] = mta_data_daily_station['TOTAL_DELTA'] / 1000

mta_data_daily_station.head()

,STATION,GTFS Latitude,GTFS Longitude,DATE,ENTRIES_DELTA,EXITS_DELTA,TOTAL_DELTA,circ_value
0,1 AV,40.731,-73.982,2019-03-30,15339.0,17243.0,32582.0,32.582
1,1 AV,40.731,-73.982,2019-03-31,11278.0,12489.0,23767.0,23.767
2,1 AV,40.731,-73.982,2019-04-01,19763.0,22034.0,41797.0,41.797
3,1 AV,40.731,-73.982,2019-04-02,19827.0,20796.0,40623.0,40.623
4,1 AV,40.731,-73.982,2019-04-03,19108.0,20330.0,39438.0,39.438


## Static Map - All Stations

In [38]:
mta_data_avg_station = (mta_data_daily
                          .groupby(['STATION','GTFS Latitude','GTFS Longitude','DATE'])['TOTAL_DELTA']
                          .sum()
                          .reset_index()
                          .groupby(['STATION','GTFS Latitude','GTFS Longitude'])['TOTAL_DELTA']
                          .mean()
                          .reset_index()
                         )

mta_data_avg_station['circ_value'] = mta_data_avg_station['TOTAL_DELTA'] / 1000 * 2
mta_data_avg_station['bin'] = pd.cut(mta_data_avg_station['circ_value'], bins=5, labels=False)
mta_data_avg_station.head()

,STATION,GTFS Latitude,GTFS Longitude,TOTAL_DELTA,circ_value,bin
0,1 AV,40.731,-73.982,24245.448,48.491,0
1,103 ST,40.791,-73.947,19738.855,39.478,0
2,103 ST,40.796,-73.961,7239.887,14.480,0
3,103 ST,40.799,-73.968,16775.081,33.550,0
4,103 ST-CORONA,40.750,-73.863,30740.806,61.482,0


In [39]:
bin_colors = ['#FFC300','#FF5733','#C70039','#900C3F','#581845']


static_map = folium.Map(
    location=[40.743781, -73.92401600000001],
    tiles='cartodbpositron',
    zoom_start=12
)

feature_group = folium.FeatureGroup("Locations")

for lat, lon, val, name, bin in zip(mta_data_avg_station['GTFS Latitude'],
                                 mta_data_avg_station['GTFS Longitude'],
                                 mta_data_avg_station['circ_value'],
                                 mta_data_avg_station['STATION'],
                                 mta_data_avg_station['bin']):
    
    feature_group.add_child(folium.Circle(
        location=[lat,lon],
        popup=name,
        radius=val,
        color=bin_colors[bin],
        fill=True,
        fill_color=bin_colors[bin]
    ))
    
static_map.add_child(feature_group)
static_map.save('static_map.html')
static_map

## Static Map - Top 10 Highlight

In [40]:
with open('pickle/top10.pickle', 'rb') as to_read: #run station_ranking to generate
    top10 = pickle.load(to_read) 
    
mta_data_avg_station['top10'] = mta_data_avg_station['STATION'].isin(top10).map({True : 1,
                                                                                False : 0})

In [41]:
top10.head(10)

0     34 ST-PENN STA
1    34 ST-HERALD SQ
2    GRD CNTRL-42 ST
3     14 ST-UNION SQ
4     TIMES SQ-42 ST
5              23 ST
6    42 ST-PORT AUTH
7              86 ST
8          FULTON ST
9             125 ST
Name: STATION, dtype: object

In [42]:
bin_colors_top10 = ['#A9A9A9','#900C3F']


static_map_top10 = folium.Map(
    location=[40.743781, -73.92401600000001],
    tiles='cartodbpositron',
    zoom_start=12
)

feature_group_top10 = folium.FeatureGroup("Locations")

for lat, lon, val, name, bin in zip(mta_data_avg_station['GTFS Latitude'],
                                 mta_data_avg_station['GTFS Longitude'],
                                 mta_data_avg_station['circ_value'],
                                 mta_data_avg_station['STATION'],
                                 mta_data_avg_station['top10']):
    
    feature_group_top10.add_child(folium.Circle(
        location=[lat,lon],
        popup=name,
        radius=val,
        color=bin_colors_top10[bin],
        fill=True,
        fill_color=bin_colors_top10[bin]
    ))
    
static_map_top10.add_child(feature_group_top10)

static_map_top10

## Dynamic Map - Daily

In [43]:
date_array = mta_data_daily_station['DATE'].unique()

list_of_df = []

for date in date_array:
    loopmask = ((mta_data_daily_station['DATE'] == date))
    loopdf = mta_data_daily_station[loopmask]
    big_list = []
    for index, rows in loopdf.iterrows(): 
    # Create list for the current row 
        lil_list =[rows['GTFS Latitude'], rows['GTFS Longitude'], rows['circ_value']] 
      
    # append the list to the final list 
        big_list.append(lil_list) 
    list_of_df.append(big_list)

In [44]:
print(len(list_of_df))
print(len(date_array))
date_array_str = [date.strftime("%m-%d-%y") for date in date_array]

63
63


In [45]:
time_map = folium.Map(
    location=[40.743781, -73.92401600000001],
    tiles='cartodbpositron',
    zoom_start=12
)


# Plot it on the map
hm = plugins.HeatMapWithTime(list_of_df,index=date_array_str,use_local_extrema=True)
hm.add_to(time_map)
# Display the map


time_map

## Dynamic Map - Hours

In [46]:
mta_data_intra_station_avg = (mta_data_intra
                              .groupby(['STATION','DATETIME','TIME','GTFS Latitude','GTFS Longitude'])['TOTAL_DELTA']
                              .sum()
                              .reset_index()
                              .groupby(['STATION','TIME','GTFS Latitude','GTFS Longitude'])['TOTAL_DELTA']
                              .mean()
                              .reset_index())

mta_data_intra_station_avg['circ_value'] = mta_data_intra_station_avg['TOTAL_DELTA'] / 1000 * 12

mta_data_intra_station_avg.head()

,STATION,TIME,GTFS Latitude,GTFS Longitude,TOTAL_DELTA,circ_value
0,1 AV,00:00:00,40.731,-73.982,741.051,8.893
1,1 AV,04:00:00,40.731,-73.982,3580.429,42.965
2,1 AV,08:00:00,40.731,-73.982,7283.349,87.400
3,1 AV,12:00:00,40.731,-73.982,7301.238,87.615
4,1 AV,16:00:00,40.731,-73.982,10496.127,125.954


In [47]:
time_array = mta_data_intra_station_avg['TIME'].unique()
time_array

array([datetime.time(0, 0), datetime.time(4, 0), datetime.time(8, 0),
       datetime.time(12, 0), datetime.time(16, 0), datetime.time(20, 0)],
      dtype=object)

In [48]:
list_of_df_intra = []

for time in time_array:
    loopmask = ((mta_data_intra_station_avg['TIME'] == time))
    loopdf = mta_data_intra_station_avg[loopmask]
    big_list = []
    for index, rows in loopdf.iterrows(): 
    # Create list for the current row 
        lil_list =[rows['GTFS Latitude'], rows['GTFS Longitude'], rows['circ_value']] 
      
    # append the list to the final list 
        big_list.append(lil_list) 
    list_of_df_intra.append(big_list)

In [49]:
time_array_str = [time.strftime("%H:%M") for time in time_array]

In [50]:
time_map_intra = folium.Map(
    location=[40.743781, -73.92401600000001],
    tiles='cartodbpositron',
    zoom_start=12
)


# Plot it on the map
hm_intra = plugins.HeatMapWithTime(list_of_df_intra,index=time_array_str,use_local_extrema=True)
hm_intra.add_to(time_map_intra)
# Display the map

time_map_intra.save('time_map_intra.html')

time_map_intra

## Dynamic Map - By Weekday

In [51]:
mta_data_byday_station = mta_data_daily_station[['STATION','GTFS Latitude', 'GTFS Longitude', 'DATE', 'TOTAL_DELTA']]
mta_data_byday_station['weekday'] = mta_data_byday_station['DATE'].apply(lambda date: date.weekday())
mta_data_byday_station.head()

/Users/alexanderparker/anaconda3/envs/metis/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,STATION,GTFS Latitude,GTFS Longitude,DATE,TOTAL_DELTA,weekday
0,1 AV,40.731,-73.982,2019-03-30,32582.0,5
1,1 AV,40.731,-73.982,2019-03-31,23767.0,6
2,1 AV,40.731,-73.982,2019-04-01,41797.0,0
3,1 AV,40.731,-73.982,2019-04-02,40623.0,1
4,1 AV,40.731,-73.982,2019-04-03,39438.0,2


In [52]:
mta_data_byday_station = (mta_data_byday_station
                          .groupby(['STATION','GTFS Latitude','GTFS Longitude','weekday'])['TOTAL_DELTA']
                          .mean()
                          .reset_index())

mta_data_byday_station['circ_value'] = mta_data_byday_station['TOTAL_DELTA'] / 1000

mta_data_byday_station.head()

,STATION,GTFS Latitude,GTFS Longitude,weekday,TOTAL_DELTA,circ_value
0,1 AV,40.731,-73.982,0,27371.889,27.372
1,1 AV,40.731,-73.982,1,21706.889,21.707
2,1 AV,40.731,-73.982,2,29619.571,29.620
3,1 AV,40.731,-73.982,3,28875.000,28.875
4,1 AV,40.731,-73.982,4,25022.375,25.022


In [53]:
week_array = mta_data_byday_station['weekday'].unique()
week_array

array([0, 1, 2, 3, 4, 5, 6])

In [54]:
list_of_df_byday = []

for day in week_array:
    loopmask = ((mta_data_byday_station['weekday'] == day))
    loopdf = mta_data_byday_station[loopmask]
    big_list = []
    for index, rows in loopdf.iterrows(): 
    # Create list for the current row 
        lil_list =[rows['GTFS Latitude'], rows['GTFS Longitude'], rows['circ_value']] 
      
    # append the list to the final list 
        big_list.append(lil_list) 
    list_of_df_byday.append(big_list)

In [55]:
week_array_str = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

In [56]:
time_map_byday = folium.Map(
    location=[40.743781, -73.92401600000001],
    tiles='cartodbpositron',
    zoom_start=12
)


# Plot it on the map
hm_byday = plugins.HeatMapWithTime(list_of_df_byday,index=week_array_str,use_local_extrema=True)
hm_byday.add_to(time_map_byday)
# Display the map

time_map_byday.save('time_map_byday.html')

time_map_byday